<a href="https://colab.research.google.com/github/anandmudagal/AI_Automation/blob/main/mlflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Section 1 install required packages

In [1]:
!pip install mlflow scikit-learn pandas numpy hyperopt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 73.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.8/733.8 kB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 12.7 MB/s eta 0:00:00


Import all libraries

In [2]:
import mlflow
import pandas as pd
import numpy as np
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials

Set MLFLOW tracking URI

In [9]:
mlflow.set_tracking_uri("https://037dff0811ae-10-244-6-7-5000.saci.r.killercoda.com/")
mlflow.set_experiment("Colab_Car_Loan_Fixed")

2025/06/25 13:17:37 INFO mlflow.tracking.fluent: Experiment with name 'Colab_Car_Loan_Fixed' does not exist. Creating a new experiment.


<Experiment: artifact_location='/root/mlruns/1', creation_time=1750857457959, experiment_id='1', last_update_time=1750857457959, lifecycle_stage='active', name='Colab_Car_Loan_Fixed', tags={}>

**Generate data**

In [10]:
np.random.seed(42)
n_samples = 100
data = {
    'age': np.random.randint(20, 60, n_samples),
    'income': np.random.randint(30000, 100000, n_samples),
    'credit_score': np.random.randint(600, 800, n_samples),
    'loan_amount': np.random.randint(10000, 50000, n_samples),
    'loan_approved': np.random.randint(0, 2, n_samples)
}
df = pd.DataFrame(data)
print("Dataset shape:", df.shape)
print("\nFirst 5 rows:")
print(df.head())

Dataset shape: (100, 5)

First 5 rows:
   age  income  credit_score  loan_amount  loan_approved
0   58   65222           743        10699              0
1   48   93335           689        10190              1
2   34   40965           711        20492              0
3   27   54538           659        45743              0
4   40   38110           712        16102              0


In [11]:
X = df.drop('loan_approved', axis=1)
y = df['loan_approved']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Training set size:", X_train.shape)
print("Test set size:", X_test.shape)
print("Features:", list(X.columns))

Training set size: (80, 4)
Test set size: (20, 4)
Features: ['age', 'income', 'credit_score', 'loan_amount']


**Hpyterparameter optimization**

In [12]:
def objective(params):
    with mlflow.start_run():
        # Add slight delay to ensure unique timestamps
        time.sleep(1)

        # Log parameters with slight variation if needed
        logged_params = params.copy()
        mlflow.log_params(logged_params)

        # Train model
        model = RandomForestClassifier(
            n_estimators=int(params['n_estimators']),
            max_depth=int(params['max_depth']),
            min_samples_split=int(params['min_samples_split']),
            random_state=42
        )
        model.fit(X_train, y_train)

        # Predict and evaluate
        y_pred = model.predict(X_test)
        y_proba = model.predict_proba(X_test)[:,1]

        accuracy = accuracy_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_proba)

        # Log metrics with slight variation if needed
        mlflow.log_metric("accuracy", accuracy)
        time.sleep(0.1)  # Ensure metric timestamps are different
        mlflow.log_metric("roc_auc", roc_auc)

        # Log model
        mlflow.sklearn.log_model(model, "model")

        return {'loss': -roc_auc, 'status': STATUS_OK}

In [13]:
space = {
    'n_estimators': hp.quniform('n_estimators', 10, 100, 5),
    'max_depth': hp.quniform('max_depth', 2, 20, 1),
    'min_samples_split': hp.quniform('min_samples_split', 2, 10, 1)
}
print("Search space defined:")
for param, range_info in space.items():
    print(f"- {param}: {range_info}")

Search space defined:
- n_estimators: 0 float
1   hyperopt_param
2     Literal{n_estimators}
3     quniform
4       Literal{10}
5       Literal{100}
6       Literal{5}
- max_depth: 0 float
1   hyperopt_param
2     Literal{max_depth}
3     quniform
4       Literal{2}
5       Literal{20}
6       Literal{1}
- min_samples_split: 0 float
1   hyperopt_param
2     Literal{min_samples_split}
3     quniform
4       Literal{2}
5       Literal{10}
6       Literal{1}


In [14]:
try:
    print("Starting hyperparameter optimization...")
    best = fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=10,
        trials=Trials()
    )
    print("Best parameters found:", best)
except Exception as e:
    print(f"Optimization failed: {e}")
    print("Check MLflow server logs for more details")

Starting hyperparameter optimization...
  0%|          | 0/10 [00:00<?, ?trial/s, best loss=?]

2025/06/25 13:18:33 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.

2025/06/25 13:18:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



🏃 View run trusting-mole-602 at: https://037dff0811ae-10-244-6-7-5000.saci.r.killercoda.com/#/experiments/1/runs/aa456813d3fc48cea551f222cb3c14f0

🧪 View experiment at: https://037dff0811ae-10-244-6-7-5000.saci.r.killercoda.com/#/experiments/1

 10%|█         | 1/10 [00:09<01:25,  9.54s/trial, best loss: -0.25]

2025/06/25 13:18:43 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.

2025/06/25 13:18:46 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



🏃 View run wistful-gnat-592 at: https://037dff0811ae-10-244-6-7-5000.saci.r.killercoda.com/#/experiments/1/runs/2ba63207e5894d359b0a844028ff24d5

🧪 View experiment at: https://037dff0811ae-10-244-6-7-5000.saci.r.killercoda.com/#/experiments/1

 20%|██        | 2/10 [00:14<00:57,  7.13s/trial, best loss: -0.3] 

...

🏃 View run casual-shark-450 at: https://037dff0811ae-10-244-6-7-5000.saci.r.killercoda.com/#/experiments/1/runs/2f69734ebdc5481e995cb083bc4d3ae2

🧪 View experiment at: https://037dff0811ae-10-244-6-7-5000.saci.r.killercoda.com/#/experiments/1

 80%|████████  | 8/10 [00:49<00:11,  5.77s/trial, best loss: -0.38]

2025/06/25 13:19:22 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.

2025/06/25 13:19:26 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



🏃 View run righteous-calf-23 at: https://037dff0811ae-10-244-6-7-5000.saci.r.killercoda.com/#/experiments/1/runs/cff0880e3cb6480e9eca76e3c22d6d36

🧪 View experiment at: https://037dff0811ae-10-244-6-7-5000.saci.r.killercoda.com/#/experiments/1

 90%|█████████ | 9/10 [00:54<00:05,  5.65s/trial, best loss: -0.38]

2025/06/25 13:19:28 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.

2025/06/25 13:19:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.



🏃 View run grandiose-crow-329 at: https://037dff0811ae-10-244-6-7-5000.saci.r.killercoda.com/#/experiments/1/runs/da5f4b06624647b3951763b93c233dff

🧪 View experiment at: https://037dff0811ae-10-244-6-7-5000.saci.r.killercoda.com/#/experiments/1

100%|██████████| 10/10 [01:00<00:00,  6.05s/trial, best loss: -0.38]
Best parameters found: {'max_depth': np.float64(3.0), 'min_samples_split': np.float64(3.0), 'n_estimators': np.float64(10.0)}
